Meta AI released a series of [Code Llama models](https://ai.meta.com/blog/code-llama-large-language-model-coding/). 

Using any of the model series has the following prompt generation flow.

<img src="images/meta_code_llama.png" height="30%" width="60%">

This notebook shows some cases how to prompt the LLM to generate 
Python and SQL code using the Anyscale Endpoints.

<img src="images/codellam-70b.png"  height="25%" width="50%">

In [1]:
import warnings
import os

import openai
from openai import OpenAI

from dotenv import load_dotenv, find_dotenv

Load the environment

In [2]:
_ = load_dotenv(find_dotenv()) # read local .env file
warnings.filterwarnings('ignore')
openai.api_base = os.getenv("ANYSCALE_API_BASE", os.getenv("OPENAI_API_BASE"))
openai.api_key = os.getenv("ANYSCALE_API_KEY", os.getenv("OPENAI_API_KEY"))
MODEL = os.getenv("MODEL")
print(f"Using MODEL={MODEL}; base={openai.api_base}")

Using MODEL=codellama/CodeLlama-70b-Instruct-hf; base=https://api.endpoints.anyscale.com/v1


In [4]:
# Create the OpenAI client, which can be used transparently with Anyscale 
# Endpoints too

from openai import OpenAI

client = OpenAI(
    api_key = openai.api_key,
    base_url = openai.api_base
)

In [5]:
# Utility function to send and fetch response

def get_commpletion(clnt: object, model: str, system_content: str, user_content:str) -> str:
    chat_completion = clnt.chat.completions.create(
        model=model,
    messages=[{"role": "system", "content": system_content},
              {"role": "user", "content": user_content}],
    temperature = 0.8)

    response = chat_completion.choices[0].message.content
    return response

In [10]:
BOLD_BEGIN = "\033[1m"
BOLD_END = "\033[0m"

#### Example 1: Generate a Python code to compute the value of PI

Use the [CO-STAR](https://towardsdatascience.com/how-i-won-singapores-gpt-4-prompt-engineering-competition-34c195a93d41) framework for prompting

1. **Context** - provide the background
2. **Objective** (or Task) - define the task to be performed
3. **Style** - instruct a writing style. Kind of sentences; formal, informal, magazine sytle, colloqiual, or allude to a know style.
4. **Audience** - who's it for?
5. **Response** - format, Text, Python, SQL, JSON, etc 


In [6]:
system_content = """You are supreme repository of knowledge, and assistant
code Co-pilot for developer to assist them in generating sample code, inclding
in langauges such as Python, JavaScript, Java, C, C++, and shell script.
"""

In [23]:
user_prompt = """
# CONTEXT #
I want to generate Python code to showcase our product feature for
serving open source large language models such as Code Llama series
on Anyscale Endpoints. The product feature is Anyscale Endpoints, which 
serves all Llama series models and the Mistral series too. For this
instance, we want to show case Code Llama 70B just released by Meta AI, and 
now hosted by Anyscale Endpoints

#############

# OBJECTIVE # 
Create a Python code to compute the value of PI using monte carlo method.

#############

# STYLE #
Use the Google style of formating Python code.

#############

# AUDIENCE #
Your code should be well commented and aimed at both beginner and
intermediate Python programers. 

#############

# RESPONSE #
Generate the entire code that can be easily copy and pasted into file: 'compute_pi.py'. Also,
provide specific instructions how to compile the code, run it, and any 
additional python packages it needs.
"""

In [24]:
response = get_commpletion(client, MODEL, system_content, user_prompt)
print(f"\n{BOLD_BEGIN}Answer - Python code{BOLD_END}:\n {response}")


Answer - Python code:
 
```
#!/usr/bin/env python

# import necessary packages
import numpy as np
import time
from multiprocessing import Process, Pipe

# Establishes the number of points for the simulation.
num_points = 10000000

# Creates a function to calculate the number of points within 
# the unit circle.
def calculate_points(num_points):
    points = np.random.uniform(-1, 1, (num_points, 2))
    distances = np.linalg.norm(points, axis=1)
    return np.sum(distances <= 1)

# Creates a function to calculate the value of pi using the 
# number of points in the unit circle.
def calculate_pi(num_points):
    return 4 * calculate_points(num_points) / num_points

# A function to run this computation in parallel. 
# It uses multiprocessing and Pipe to connect the processes and send
# the resulting value of pi through the pipe.
def run_in_parallel(num_points):
    pipe1, pipe2 = Pipe()
    p = Process(target=calculate_pi, args=(num_points, pipe1))
    p.start()
    result = pipe2.recv()

### Example 2: Generate a Python code generate fractions

Generate Python code that a) generates fractions between 1 and 10, finds and common denominator, and computes the sum.

In [21]:
user_prompt = """
# CONTEXT #
I want to generate Python code to showcase our product feature for
serving open source large language models such as Code Llama series
on Anyscale Endpoints. The product feature is Anyscale Endpoints, which 
serves all Llama series models and the Mistral series too. For this
instance, we want to show case Code Llama 70B just released by Meta AI, and 
now hosted by Anyscale Endpoints

#############

# OBJECTIVE # 
Create a Python code that generates a list of fractions between 1 and 10.
It then generates a common denominator, and adds the sum of all fractions.

#############

# STYLE #
Use the Google style of formating Python code.

#############

# AUDIENCE #
Your code should be well commented and aimed at both beginner and
intermediate Python programers. 

#############

# RESPONSE #
Generate the entire code that can be easily copy and pasted into file: 'add_fractions.py'. Also,
provide specific instructions how to compile the code, run it, and any 
additional python packages it needs. The final output of the code
should be a list of fractions genetated, their sum, and their common denominator.
"""

In [22]:
response = get_commpletion(client, MODEL, system_content, user_prompt)
print(f"\n{BOLD_BEGIN}Answer - Python code{BOLD_END}:\n {response}")


Answer - Python code:
 1. Install dependencies:
    * We will use the `fractions` module in Python 3 to generate fractions. Run the following command to install the module:
    ```
    pip install fractions
    ```
2. Create a file named `add_fractions.py`.
3. Copy and paste the following code:

```python
# add_fractions.py

from fractions import Fraction

# Create a list of fractions between 1 and 10
fractions = [Fraction(i, 10) for i in range(1, 10)]

# Calculate the sum of all fractions
sum_fractions = sum(fractions, 0)

# Calculate the common denominator
common_denominator = sum_fractions.denominator

# Print the result
print("Fractions:")
for f in fractions:
    print(f" - {f}")
print()
print(f"Sum: {sum_fractions}")
print(f"Common Denominator: {common_denominator}")
```

4. Run the code using the command:

```
python add_fractions.py
```

Example output:

```
$ python add_fractions.py
Fractions:
 - 1/10
 - 1/5
 - 3/10
 - 2/5
 - 1/2
 - 3/5
 - 7/10
 - 4/5
 - 9/10

Sum: 59/50
Commo